In [421]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(r"C:\Users\Minji Kang\Documents\GitHub\network_reliability\BNS-JT-python")

import re 
import importlib
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import json
from gurobipy import Model, GRB, quicksum

import BNS_JT.brc as brc
import BNS_JT.cpm as cpm
import BNS_JT.variable as variable
import BNS_JT.operation as operation
import BNS_JT.branch as branch

importlib.reload(brc)
importlib.reload(cpm)
importlib.reload(variable)
importlib.reload(operation)
importlib.reload(branch)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<module 'BNS_JT.branch' from 'C:\\Users\\Minji Kang\\Documents\\GitHub\\network_reliability\\BNS-JT-python\\BNS_JT\\branch.py'>

# Network Topology
Generate Random Arc Capacity Value

In [422]:
# Node and edge data
nodes = {
    "n1": (0, 0),
    "n2": (10, 10),
    "n3": (10, -10),
    "n4": (20, 0)
}

edges = {  
    "e1": ("n1", "n2"),   
    "e2": ("n2", "n1"),   
    "e3": ("n1", "n3"),   
    "e4": ("n3", "n1"),   
    "e5": ("n2", "n3"),   
    "e6": ("n3", "n2"),   
    "e7": ("n2", "n4"),   
    "e8": ("n4", "n2"),   
    "e9": ("n3", "n4"),   
    "e10": ("n4", "n3")   
}

arcs = [(u, v) for _, (u, v) in edges.items()]

# Probability of failure/survival (failure: 0, survival: 1)
probs = {
    'e1': {0: 0.01, 1: 0.99}, 
    'e2': {0: 0.01, 1: 0.99}, 
    'e3': {0: 0.01, 1: 0.99},
    'e4': {0: 0.01, 1: 0.99},
    'e5': {0: 0.05, 1: 0.95},
    'e6': {0: 0.05, 1: 0.95}, 
    'e7': {0: 0.05, 1: 0.95}, 
    'e8': {0: 0.05, 1: 0.95},
    'e9': {0: 0.10, 1: 0.90}, 
    'e10': {0: 0.10, 1: 0.90}
}

# Initial intact capacity
intact_capacity = { 
    "e1": 15, "e2": 15, 
    "e3": 15, "e4": 15,
    "e5": 7.5, "e6": 7.5,
    "e7": 15, "e8": 15, 
    "e9": 15, "e10": 15
}

# Function to generate random component states (0 or 1) based on failure probabilities
def generate_comps_st(probs):
    comps_st = {}

    # Define edge pairs that should have identical states
    edge_pairs = [("e1", "e2"), ("e3", "e4"), ("e5", "e6"), ("e7", "e8"), ("e9", "e10")]

    # Process edge pairs
    for e1, e2 in edge_pairs:
        state = np.random.choice([0, 1], p=[probs[e1][0], probs[e1][1]])  # Sample once
        comps_st[e1] = state
        comps_st[e2] = state  # Assign the same state to the paired edge

    return comps_st

comps_st = generate_comps_st(probs)
arc_capacity = {edge: intact_capacity[edge] * comps_st[edge] for edge in intact_capacity}

print("Generated Component States:")
for edge, state in comps_st.items():
    print(f"{edge}: {state}")

# Compute distances
def euclidean_distance(node1, node2):
    x1, y1 = nodes[node1]
    x2, y2 = nodes[node2]
    return round(((x2 - x1)**2 + (y2 - y1)**2)**0.5, 2)

arc_distance = {edge_name: euclidean_distance(u, v) for edge_name, (u, v) in edges.items()}   

# Create the graph
G = nx.DiGraph()

for node, position in nodes.items():
    G.add_node(node, pos=position)

for edge_name, (u, v) in edges.items(): 
    distance = arc_distance[edge_name]   
    capacity = arc_capacity[edge_name]  
    G.add_edge(u, v, weight=distance, capacity=capacity)   

# Calculate maximum allowable distance
# Demand data
demand = {
    'k1': {'origin': 'n1', 'destination': 'n4', 'amount': 28},
    'k2': {'origin': 'n2', 'destination': 'n3', 'amount': 18},
    'k3': {'origin': 'n1', 'destination': 'n3', 'amount': 11},
    'k4': {'origin': 'n3', 'destination': 'n4', 'amount': 16},
    'k5': {'origin': 'n4', 'destination': 'n1', 'amount': 20},
    'k6': {'origin': 'n3', 'destination': 'n2', 'amount': 14},
    'k7': {'origin': 'n3', 'destination': 'n1', 'amount': 11},
    'k8': {'origin': 'n4', 'destination': 'n3', 'amount': 18}
}

# Maximum allowable delay time 6 minutes / average velocity 149 km/h
avg_velo = 149
max_distance = {}

for commodity, info in demand.items():
    shortest_distance = nx.shortest_path_length(G, source=info['origin'], target=info['destination'], weight='weight')
    max_allowable_time = (shortest_distance * 60) / avg_velo + 6  # 6 minutes extra
    max_distance[commodity] = max_allowable_time * avg_velo / 60  # Store per commodity

    print(f"\nCommodity: {commodity}")
    print(f"  Shortest distance: {shortest_distance} km")
    print(f"  Maximum allowable time: {max_allowable_time:.2f} minutes")
    print(f"  Maximum allowable distance: {max_distance[commodity]:.2f} km")


# Plot the network
plt.figure(figsize=(8, 6))
pos = nx.get_node_attributes(G, 'pos')
nx.draw_networkx_nodes(G, pos, node_size=450, node_color="lightblue")
nx.draw_networkx_edges(
    G, pos,
    edgelist=list(edges.values()),  
    arrowstyle='->',
    arrowsize=15,
    connectionstyle='arc3,rad=0.1',
    min_target_margin=10,
    min_source_margin=10
)
edge_labels = {(u, v): f"{arc_distance[edge_name]} km, Cap: {arc_capacity[edge_name]}" 
               for edge_name, (u, v) in edges.items()}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8, label_pos=0.5)
nx.draw_networkx_labels(G, pos, font_size=9)
plt.title("Network with Bidirectional Arcs")
plt.show()

# Print edge information
print("\nEdge Information:")
for edge_name, (u, v) in edges.items():
    edge_data = G.get_edge_data(u, v) 
    capacity = edge_data['capacity']
    print(f"{edge_name} Capacity: {capacity}")

Generated Component States:
e1: 1
e2: 1
e3: 1
e4: 1
e5: 1
e6: 1
e7: 1
e8: 1
e9: 1
e10: 1

Commodity: k1
  Shortest distance: 28.28 km
  Maximum allowable time: 17.39 minutes
  Maximum allowable distance: 43.18 km

Commodity: k2
  Shortest distance: 20.0 km
  Maximum allowable time: 14.05 minutes
  Maximum allowable distance: 34.90 km

Commodity: k3
  Shortest distance: 14.14 km
  Maximum allowable time: 11.69 minutes
  Maximum allowable distance: 29.04 km

Commodity: k4
  Shortest distance: 14.14 km
  Maximum allowable time: 11.69 minutes
  Maximum allowable distance: 29.04 km

Commodity: k5
  Shortest distance: 28.28 km
  Maximum allowable time: 17.39 minutes
  Maximum allowable distance: 43.18 km

Commodity: k6
  Shortest distance: 20.0 km
  Maximum allowable time: 14.05 minutes
  Maximum allowable distance: 34.90 km

Commodity: k7
  Shortest distance: 14.14 km
  Maximum allowable time: 11.69 minutes
  Maximum allowable distance: 29.04 km

Commodity: k8
  Shortest distance: 14.14 km


In [423]:
varis = {}
for k, v in edges.items():
    varis[k] = variable.Variable( name=k, values = [0, 1]) # values: edge flow capacity

### Demand required for the network topology

In [424]:
# Compute total demand separately for each edge 
edge_demand = {edge: 0 for edge in edges.keys()}

# Create mapping for bidirectional edges 
bidirectional_mapping = {}
edge_list = list(edges.keys())

for i in range(0, len(edge_list), 2):
    if i + 1 < len(edge_list):
        bidirectional_mapping[edge_list[i]] = edge_list[i + 1]
        bidirectional_mapping[edge_list[i + 1]] = edge_list[i]

# Sum up demand for edges used in any shortest path (all possible shortest paths)
for demand_key, info in demand.items():
    origin, destination, amount = info["origin"], info["destination"], info["amount"]
    try:
        all_paths = list(nx.all_shortest_paths(G, source=origin, target=destination, weight="weight"))
        
        for path in all_paths:
            for i in range(len(path) - 1):
                path_edge = (path[i], path[i + 1])
                for edge_name, (u, v) in edges.items():
                    if path_edge == (u, v):  # Forward direction
                        edge_demand[edge_name] += amount  
                    elif path_edge == (v, u):  # Backward direction
                        reverse_edge = bidirectional_mapping.get(edge_name, edge_name)
                        edge_demand[reverse_edge] += amount  
    except nx.NetworkXNoPath:
        pass  # Ignore cases where no path exists

# Normalize demand values with a small shift (epsilon) to prevent 0
D_max = max(edge_demand.values()) if max(edge_demand.values()) > 0 else 1  # Prevent division by zero
D_min = min(edge_demand.values()) if min(edge_demand.values()) > 0 else 0
epsilon = 0.1  # Small shift to prevent 0 values

normalized_demand = {
    edge: (demand - D_min + epsilon) / (D_max - D_min + epsilon) if D_max != D_min else 0.5
    for edge, demand in edge_demand.items()
}

# Create custom colormap
cmap = mpl.colors.LinearSegmentedColormap.from_list("gray_to_blue", ["lightgray", "blue"])

# Create NetworkX graph
G = nx.DiGraph()
for node, position in nodes.items():
    G.add_node(node, pos=position)
for edge_name, (u, v) in edges.items():
    G.add_edge(u, v, weight=arc_distance[edge_name], label=edge_name)
pos = nx.get_node_attributes(G, "pos")

# Create figure
fig, ax = plt.subplots(figsize=(8, 5))
ax.set_frame_on(True)  # Ensure plot box is visible
nx.draw_networkx_nodes(G, pos, node_size=30, node_color="black", ax=ax)
drawn_edges = set()  # To avoid duplicate plotting

for edge_name, (u, v) in edges.items():
    if edge_name in drawn_edges:
        continue  # Skip if already drawn

    color = cmap(normalized_demand[edge_name])

    # Forward direction (u -> v)
    nx.draw_networkx_edges(
        G, pos, edgelist=[(u, v)], width=2,
        edge_color=[color], edge_cmap=cmap,
        arrowstyle="->", arrowsize=10,
        alpha=1, ax=ax, connectionstyle="arc3,rad=0.08"
    )

    # Reverse direction (v -> u)
    reverse_edge = bidirectional_mapping.get(edge_name, edge_name)
    if reverse_edge != edge_name:  # Ensure backward edge exists
        color_reverse = cmap(normalized_demand[reverse_edge])
        nx.draw_networkx_edges(
            G, pos, edgelist=[(v, u)], width=2,
            edge_color=[color_reverse], edge_cmap=cmap,
            arrowstyle="->", arrowsize=10,
            alpha=1, ax=ax, connectionstyle="arc3,rad=0.08"
        )

    # Mark both forward and backward edges as drawn
    drawn_edges.add(edge_name)
    drawn_edges.add(reverse_edge)

# Colorbar setup
sm = mpl.cm.ScalarMappable(cmap=cmap, norm=mpl.colors.Normalize(vmin=D_min, vmax=D_max))
sm.set_array([])
plt.colorbar(sm, ax=ax, label="Edge Demand Intensity")
plt.title("Demand required for the network topology")
plt.show()

# MCNF System Function

By using Gurobi solver

연결성만으로 시스템이 생존(sys_st = 's')한다고 평가하지 않고, 기대 손실(expected_loss)이 기준 이하일 때만 생존으로 간주
그렇다면 최소 생존 상태(min_comps_st)는 단순히 "연결만 되는 상태"를 의미하는 것이 아니라, "손실을 최소화하면서 운영이 가능한 상태"여야 함

In [425]:
def MCNF_systemfunc(arcs, comps_st, edges, arc_capacity, demand, max_distance, arc_distance):
    from gurobipy import Model, GRB, quicksum
    import networkx as nx

    # Create a graph for shortest path calculations
    G = nx.Graph()
    for e, (i, j) in edges.items():
        G.add_edge(i, j, weight=arc_distance.get(e, 1))  # Set edge weights based on distance

    # Create Gurobi optimization model
    model = Model("Network Flow Optimization")
    model.setParam('OutputFlag', 0) 

    # Define variables
    flow = {}
    unmet_demand = {}

    for k, info in demand.items():
        unmet_demand[k] = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"unsatisfied_{k}")
        for i, j in arcs:
            arc_key = next((e for e, v in edges.items() if v == (i, j) or v == (j, i)), None)
            capacity = arc_capacity.get(arc_key, 0)
            flow[k, i, j] = model.addVar(lb=0, ub=capacity, vtype=GRB.CONTINUOUS, name=f"flow_{k}_{i}_{j}")

    # Objective function: Minimize expected loss
    cost_coefficient = {
        k: info['amount'] * nx.shortest_path_length(G, source=info['origin'], target=info['destination'], weight='weight') * 0.1723
        for k, info in demand.items()
    }
    
    model.setObjective(
        quicksum(cost_coefficient[k] * unmet_demand[k] for k in demand),
        GRB.MINIMIZE
    )

    # Extract all nodes from edge values
    nodes = set(node for edge in edges.values() for node in edge)

    # Constraint 1: Flow conservation
    for k, info in demand.items():
        origin = info['origin']
        destination = info['destination']
        amount = info['amount']
        for node in nodes: 
            inflow = quicksum(flow[k, i, j] for i, j in arcs if j == node)
            outflow = quicksum(flow[k, i, j] for i, j in arcs if i == node)
            if node == origin:
                model.addConstr(outflow - inflow == amount - unmet_demand[k])
            elif node == destination:
                model.addConstr(outflow - inflow == - amount + unmet_demand[k])
            else:
                model.addConstr(outflow - inflow == 0)

    # Constraint 2: Arc capacity limits
    for i, j in arcs:
        arc_key = next((e for e, v in edges.items() if v == (i, j) or v == (j, i)), None)
        model.addConstr(quicksum(flow[k, i, j] for k in demand if (k, i, j) in flow) <= arc_capacity.get(arc_key, 0))

    # Constraint 3: Commodity별 max_distance 적용
    for k, info in demand.items():
        origin = info['origin']
        distance_expr = quicksum(arc_distance.get(e, 0) * flow[k, i, j] for e, (i, j) in edges.items() if (k, i, j) in flow)
        total_flow = quicksum(flow[k, i, j] for i, j in arcs if (k, i, j) in flow and i == origin)
        
        # max_distance를 dictionary에서 해당 commodity에 맞게 참조
        model.addConstr(distance_expr <= max_distance[k] * total_flow)

    # Perform optimization
    model.optimize()

    # Process results
    if model.status == GRB.OPTIMAL:
        expected_loss = model.objVal
     
        if expected_loss < 2400:
            sys_st = 's'

        else:
            sys_st = 'f'

        # Minimum survival rule extraction
        min_comps_st = None
 
        #print(f"Expected Loss: {expected_loss}, System State: {sys_st}")
        
        return expected_loss, sys_st, min_comps_st  

    else:
        return None, None, None  

Run MCNF system function

In [426]:
sys_fun = lambda comps_st: MCNF_systemfunc(
    arcs=arcs,
    comps_st=comps_st, 
    edges=edges,
    arc_capacity={edge: intact_capacity[edge] * comps_st[edge] for edge in intact_capacity},  
    demand=demand,
    max_distance=max_distance, 
    arc_distance=arc_distance
)


# Print input values
print("\n🔹 Input Values:")
print("Component States (comps_st):", comps_st)
print("Edges:", edges)
print("Arc Capacity:", arc_capacity)
print("Demand:", demand)
print("Max Distance:", max_distance)
print("Arc Distance:", arc_distance)

# Run the function and capture outputs
expected_loss, sys_st, min_comps_st = sys_fun(comps_st)

# Print output values
print("\n🔹 Output Values:")
print("System State:", sys_st)
print("Minimum component state:", min_comps_st) 


🔹 Input Values:
Component States (comps_st): {'e1': np.int64(1), 'e2': np.int64(1), 'e3': np.int64(1), 'e4': np.int64(1), 'e5': np.int64(1), 'e6': np.int64(1), 'e7': np.int64(1), 'e8': np.int64(1), 'e9': np.int64(1), 'e10': np.int64(1)}
Edges: {'e1': ('n1', 'n2'), 'e2': ('n2', 'n1'), 'e3': ('n1', 'n3'), 'e4': ('n3', 'n1'), 'e5': ('n2', 'n3'), 'e6': ('n3', 'n2'), 'e7': ('n2', 'n4'), 'e8': ('n4', 'n2'), 'e9': ('n3', 'n4'), 'e10': ('n4', 'n3')}
Arc Capacity: {'e1': np.int64(15), 'e2': np.int64(15), 'e3': np.int64(15), 'e4': np.int64(15), 'e5': np.float64(7.5), 'e6': np.float64(7.5), 'e7': np.int64(15), 'e8': np.int64(15), 'e9': np.int64(15), 'e10': np.int64(15)}
Demand: {'k1': {'origin': 'n1', 'destination': 'n4', 'amount': 28}, 'k2': {'origin': 'n2', 'destination': 'n3', 'amount': 18}, 'k3': {'origin': 'n1', 'destination': 'n3', 'amount': 11}, 'k4': {'origin': 'n3', 'destination': 'n4', 'amount': 16}, 'k5': {'origin': 'n4', 'destination': 'n1', 'amount': 20}, 'k6': {'origin': 'n3', 'des

# Expected Loss Evaluation
By BRC algorithm

In [427]:
# Run BRC
brs, rules, sys_res, monitor = brc.run(varis, probs, sys_fun, max_sf=np.inf, max_nb=np.inf, pf_bnd_wr=0.0, brs=[])

# System failure probability
P_f = sum(branch.p for branch in brs if branch.down_state == 'f')
print(f"System Failure Probability (P_f): {P_f}")

# Print survival rules
print("\nAll Survival Rules:")
for i, rule in enumerate(rules['s'], 1):
    print(f"Rule {i}: {rule}")

# Print failure rules
print("\nAll Failure Rules:")
for i, rule in enumerate(rules['f'], 1):
    print(f"Rule {i}: {rule}")
    
# Print results
print("\n\n🔹 All Branches and Expected Loss:")
for i, branch in enumerate(brs, start=1):  
    expected_loss_down, _, _ = sys_fun(branch.down) 
    expected_loss_up, _, _ = sys_fun(branch.up)

    print(f"\nBranch {i}:")
    print(f"  Down State: {branch.down}")
    print(f"  Up State: {branch.up}")
    print(f"  Probability: {branch.p}")
    print(f"  Expected Loss (Down State): {expected_loss_down:.4f} / {branch.down_state}")
    print(f"  Expected Loss (Up State): {expected_loss_up:.4f} / {branch.up_state}\n\n\n")

*** Analysis completed with f_sys runs 37: out_flag = complete ***
The # of found non-dominated rules (f, s): 6 (5, 1)
Probability of branchs (f, s, u): (2.0391e-01, 7.96e-01, 2.2204e-16)
The # of branches (f, s, u), (min, avg) len of rf: 6 (5, 1, 0), (1, 1.00)
System Failure Probability (P_f): 0.203913775

All Survival Rules:
Rule 1: {'e1': 1, 'e3': 1, 'e5': 1, 'e7': 1, 'e9': 1}

All Failure Rules:
Rule 1: {'e9': 0}
Rule 2: {'e5': 0}
Rule 3: {'e7': 0}
Rule 4: {'e1': 0}
Rule 5: {'e3': 0}


🔹 All Branches and Expected Loss:

Branch 1:
  Down State: {'e1': 1, 'e2': 0, 'e3': 1, 'e4': 0, 'e5': 1, 'e6': 0, 'e7': 1, 'e8': 0, 'e9': 0, 'e10': 0}
  Up State: {'e1': 1, 'e2': 1, 'e3': 1, 'e4': 1, 'e5': 1, 'e6': 1, 'e7': 1, 'e8': 1, 'e9': 0, 'e10': 1}
  Probability: 0.08845402499999999
  Expected Loss (Down State): 4638.8536 / f
  Expected Loss (Up State): 4638.8536 / f




Branch 2:
  Down State: {'e1': 1, 'e2': 0, 'e3': 1, 'e4': 0, 'e5': 1, 'e6': 0, 'e7': 1, 'e8': 0, 'e9': 1, 'e10': 0}
  Up Stat

# Component Importance Calculation

### Birnbaum's Importance Measure (BM)

The **Birnbaum Importance Measure (BM)** quantifies the sensitivity of system reliability $p_s$ to the reliability of a specific component $p_i$. It is defined as:

$$
BM_i = P(S = 1 \mid X_i = 1) - P(S = 1 \mid X_i = 0)
$$

where:

- $P(S = 1 \mid X_i = 1)$ is the probability that the system functions given that component $X_i$ is functional.
- $P(S = 1 \mid X_i = 0)$ is the probability that the system functions given that component $X_i$ has failed.

This measure helps in assessing how much the reliability of an individual component influences the overall system reliability.


In [428]:
# Dictionary to store probabilities for each component
component_survival_probs = {}
component_CIM = {}  

# 1. Select only branches where the system survives ('s')
B_s = [branch for branch in brs if branch.down_state == 's']

# 2. Compute P(S=1, X_i=1) for each component
for x_i in comps_st.keys():
    target_prob1 = 0  
    expression_terms = []  
    detailed_calculations = []  

    for branch in B_s:
        l_states = branch.down  # Lower state (component failures)
        u_states = branch.up    # Upper state (component survival)
        p_branch = branch.p     # Branch probability

        # Case 1: down state = 1 (X_i = 1)
        if l_states.get(x_i, 1) == 1:
            target_prob1 += p_branch  # Add P(branch)
            expression_terms.append(f"{p_branch:.6f}")

        # Case 2: down state = 0 and up state = 1 (X_i = 1)
        elif l_states.get(x_i, 1) == 0 and u_states.get(x_i, 1) == 1:
            term = p_branch * probs[x_i][1]  # P(branch) * P(X_i=1)
            target_prob1 += term

    # Store the result for P(S=1, X_i=1)
    component_survival_probs[x_i] = target_prob1

    # Compute CIM (P(S=1 | X_i=1) = P(S=1, X_i=1) / P(X_i=1))
    CIM = target_prob1 / probs[x_i][1]  
    component_CIM[x_i] = CIM  



# Dictionary to store probabilities for each component
component_survival_probs_0 = {}
component_CIM_0 = {}  

# 2. Compute P(S=1, X_i=0) for each component
for x_i in comps_st.keys():
    target_prob2 = 0  
    expression_terms = []  # Store terms for printing
    detailed_calculations = []  # Store detailed multiplication steps

    for branch in B_s:
        l_states = branch.down  
        u_states = branch.up    
        p_branch = branch.p     

        # Case 1: up_state = 0 (X_i = 0)
        if u_states.get(x_i, 1) == 0:
            target_prob2 += p_branch  # Add P(branch)

        # Case 2: down_state = 0 and up_state = 1 (X_i initially fails but later survives)
        elif l_states.get(x_i, 1) == 0 and u_states.get(x_i, 1) == 1:
            term = p_branch * probs[x_i][0]  # P(branch) * P(X_i=0)
            target_prob2 += term

    # Store the result for P(S=1, X_i=0)
    component_survival_probs_0[x_i] = target_prob2

    # Compute CIM (P(S=1 | X_i=0) = P(S=1, X_i=0) / P(X_i=0))
    CIM_0 = target_prob2 / probs[x_i][0]  
    component_CIM_0[x_i] = CIM_0  # Store CIM



# Dictionary to store BM values
component_BM = {}

# Compute BM for each component
print("\n🔹 Final Results for Birnbaum's Importance Measure (BM):")
for component in comps_st.keys():
    BM_value = component_CIM[component] - component_CIM_0[component]  # BM = CIM - CIM_0

    # Fix small negative values (replace -0.000000 with 0.000000)
    if abs(BM_value) < 1e-10:
        BM_value = 0.0  # Set very small values to exact 0

    component_BM[component] = BM_value  # Store BM

    # Print BM calculation
    print(f"  BM({component}) = {component_CIM[component]:.6f} - {component_CIM_0[component]:.6f} = {BM_value:.6f}")



🔹 Final Results for Birnbaum's Importance Measure (BM):
  BM(e1) = 0.804127 - 0.000000 = 0.804127
  BM(e2) = 0.796086 - 0.796086 = 0.000000
  BM(e3) = 0.804127 - 0.000000 = 0.804127
  BM(e4) = 0.796086 - 0.796086 = 0.000000
  BM(e5) = 0.837985 - 0.000000 = 0.837985
  BM(e6) = 0.796086 - 0.796086 = 0.000000
  BM(e7) = 0.837985 - 0.000000 = 0.837985
  BM(e8) = 0.796086 - 0.796086 = 0.000000
  BM(e9) = 0.884540 - 0.000000 = 0.884540
  BM(e10) = 0.796086 - 0.796086 = 0.000000


In [429]:
# Function to extract numbers from component names for proper sorting
def extract_number(component_name):
    match = re.search(r'\d+', component_name)  # Find numeric part
    return int(match.group()) if match else float('inf')  # Convert to int

# Sort components numerically instead of lexicographically
sorted_components = sorted(component_BM.keys(), key=extract_number)
BM_values = [component_BM[comp] for comp in sorted_components]

# Plot bar chart
plt.figure(figsize=(8, 5))
plt.bar(sorted_components, BM_values, color='gray', alpha=0.7)
plt.xlabel("Component")
plt.ylabel("Birnbaum's Importance Measure (BM)")
plt.title("Component Importance Based on Birnbaum's Measure")
plt.xticks(rotation=45)  # Rotate x-axis labels for readability
plt.ylim(0, 1)  # Y-axis range fixed from 0 to 1
plt.grid(axis="y", linestyle="--", alpha=0.5)

# Show plot
plt.show()


In [430]:
# Create custom colormap from light gray to red
cmap = mpl.colors.LinearSegmentedColormap.from_list("gray_to_red", ["lightgray", "red"])

# Create NetworkX graph
G = nx.DiGraph()
for node, position in nodes.items():
    G.add_node(node, pos=position)
for edge_name, (u, v) in edges.items():
    G.add_edge(u, v, weight=arc_distance[edge_name], label=edge_name)
pos = nx.get_node_attributes(G, "pos")

# Get min and max BM values for color scaling
BM_max = max(component_BM.values()) if max(component_BM.values()) > 0 else 1
BM_min = min(component_BM.values()) if min(component_BM.values()) > 0 else 0

# Create figure
fig, ax = plt.subplots(figsize=(8, 5))
ax.set_frame_on(True)  # Ensure plot box is visible
nx.draw_networkx_nodes(G, pos, node_size=30, node_color="black", ax=ax)
drawn_edges = set()  # To avoid duplicate plotting

for edge_name, (u, v) in edges.items():
    if edge_name in drawn_edges:
        continue  # Skip if already drawn

    # Map BM values to color scale (without normalization)
    color = cmap((component_BM[edge_name] - BM_min) / (BM_max - BM_min) if BM_max != BM_min else 0.5)

    # Forward direction (u -> v)
    nx.draw_networkx_edges(
        G, pos, edgelist=[(u, v)], width=2,
        edge_color=[color], edge_cmap=cmap,
        arrowstyle="->", arrowsize=10,
        alpha=1, ax=ax, connectionstyle="arc3,rad=0.08"
    )

    # Reverse direction (v -> u)
    reverse_edge = bidirectional_mapping.get(edge_name, edge_name)
    if reverse_edge != edge_name:  # Ensure backward edge exists
        color_reverse = cmap((component_BM[reverse_edge] - BM_min) / (BM_max - BM_min) if BM_max != BM_min else 0.5)
        nx.draw_networkx_edges(
            G, pos, edgelist=[(v, u)], width=2,
            edge_color=[color_reverse], edge_cmap=cmap,
            arrowstyle="->", arrowsize=10,
            alpha=1, ax=ax, connectionstyle="arc3,rad=0.08"
        )

    # Mark both forward and backward edges as drawn
    drawn_edges.add(edge_name)
    drawn_edges.add(reverse_edge)

# Colorbar setup
sm = mpl.cm.ScalarMappable(cmap=cmap, norm=mpl.colors.Normalize(vmin=BM_min, vmax=BM_max))
sm.set_array([])
plt.colorbar(sm, ax=ax, label="Birnbaum's Importance Measure (BM)")

plt.title("Network Topology with BM Importance")
plt.show()
